In [54]:
import pandas as pd
import numpy as np
import os

In [ ]:
def save_fpl_players_24_25(fpl_subfolders, season):
    for folder in fpl_subfolders:
        player_id = int(folder.split('/')[-1].split('_')[-1])

        if players_known[players_known['fpl_id'] == player_id]['fpl_id'].tolist():
            # print(player_id, players_known[players_known['fpl_id'] == player_id]['fpl_id'].tolist(), players_known[players_known['fpl_id'] == player_id]['fpl_id'].tolist()[0])
            player_df = pd.read_csv(str(folder+'/gw.csv'))
            player_df['fpl_id'] = players_known[players_known['fpl_id'] == player_id]['fpl_id'].tolist()[0]
            player_dir = './data/joint/'+str(season)+'/fpl/'

            # Check if player_dir exists
            if not os.path.exists(player_dir):
                os.makedirs(player_dir)
            player_df.to_csv(player_dir + str(player_id) + '.csv', index_label=False)


    print(' successfully cleaned 20'+ season +' fpl data')

# Understat Files
def save_under_players_24_25(understat_files, season):
    for file_ in understat_files:
        player_id = file_.split('/')[-1].split('_')[-1].split('.csv')[0]
        player_df = pd.read_csv(str(file_))
        player_df['understat_id'] = player_id
        player_dir = './data/joint/'+str(season)+'/understat/'

        # Check if player_dir exists
        if not os.path.exists(player_dir):
            os.makedirs(player_dir)
        player_df.to_csv(player_dir + str(player_id) + '.csv', index_label=False)

    print('successfully cleaned understat 20'+ season +' data')

def joint_players_info_24_25(fpl_player_folder_path, understat_player_folder_path, season):
    fpl_subfolders = [ f.path for f in os.scandir(fpl_player_folder_path) if f.is_dir() ]
    under_files = [ f.path for f in os.scandir(understat_player_folder_path) if f.is_file() ]
    save_fpl_players_24_25(fpl_subfolders, season)
    save_under_players_24_25(under_files, season)

    print('20'+ season +' fpl and understat data now in `joint` folder')



In [ ]:
def merge_fpl_understat_data_24_25(fpl_player_folder_path, understat_player_folder_path, season):
    joint_players_info_24_25(fpl_player_folder_path, understat_player_folder_path, season)

    all_players_details = pd.read_csv('./fpl_understat_id_name.csv') # From ./player_data.ipynbs

    joint_fpl_data_path = "./data/joint/"+ season + "/fpl/"
    joint_understat_path = "./data/joint/"+ season + "/understat/"

    understat_files = next(os.walk(joint_understat_path), (None, None, []))[2]  # [] if no file
    fpl_files = next(os.walk(joint_fpl_data_path), (None, None, []))[2]  # [] if no file

    understat_names = [file_.split('.')[0] for file_ in understat_files]
    fpl_file_names = [file_.split('.')[0] for file_ in fpl_files]

    for fpl_id in fpl_file_names:
        player_details_ = all_players_details[all_players_details['fpl_id'] == int(fpl_id)]
        under_id = player_details_['understat_id'].values[0]

        fpl_player = pd.read_csv(joint_fpl_data_path + fpl_id + '.csv')
        understat_player = pd.read_csv(joint_understat_path + str(under_id) + '.csv')

        # Change 'kickoff_time' column name to 'date
        fpl_player_data = fpl_player.rename(columns={'kickoff_time': 'date'})

        # change the formats: From 2021-10-03T13:00:00Z to 2021-10-03
        fpl_player_data.date = fpl_player_data.date.apply(lambda x: x.split('T')[0])

        # Dates are of the form 2021-10-03T13:00:00Z
        fpl_dates_min = fpl_player_data['date'].min()
        fpl_dates_max = fpl_player_data['date'].max()

        # Filter out player info not in the range of dates we are dealing with
        understat_filtered = understat_player[(pd.to_datetime(understat_player['date']) >= pd.to_datetime(fpl_dates_min))
                                                    & (pd.to_datetime(understat_player['date']) <= pd.to_datetime(fpl_dates_max) )]

        # Marge fpl_player_data with understat_player_data if the dates match
        player_data_merged = fpl_player_data.merge(understat_filtered, on="date")

        # Add player team
        def set_player_team(row):
            return row['h_team'] if row['was_home'] else row['a_team']

        with_team = player_data_merged.apply(set_player_team, axis=1)
        player_data_merged['player_team'] = player_data_merged.apply(set_player_team, axis=1)


        if(player_data_merged.shape[0]):
            merged_dir = './data/joint/'+ season +'/merged/'
            if not os.path.exists(merged_dir):
                os.makedirs(merged_dir)

            player_data_merged.to_csv(merged_dir+ fpl_id +'.csv', index_label=False )